In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
date = "2025-12"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
end_use = "naics?get=ALL_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_exports"] = df["ALL_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_exports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_exports", ascending = False).CTY_CODE)[0:31]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [4]:
df.tail()

,ALL_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL,time,total_exports
39278,6304016,7940,ZAMBIA,DET,2025-12,6304016.0
39279,5506245,7950,ESWATINI,DET,2025-12,5506245.0
39280,2091091,7960,ZIMBABWE,DET,2025-12,2091091.0
39281,3162304,7970,MALAWI,DET,2025-12,3162304.0
39282,113658,7990,LESOTHO,DET,2025-12,113658.0


In [5]:
country_list[0] = ""

In [6]:
country_list.extend(["0003", "0020"])

In [7]:
country_list

['',
 '1220',
 '2010',
 '5700',
 '5880',
 '4120',
 '4280',
 '4210',
 '5800',
 '3510',
 '4279',
 '5590',
 '5830',
 '4231',
 '5820',
 '5330',
 '4419',
 '6021',
 '4759',
 '5200',
 '3370',
 '3010',
 '4700',
 '5570',
 '5170',
 '5081',
 '5490',
 '4890',
 '4190',
 '3330',
 '5520',
 '0003',
 '0020']

In [9]:
end_use = "hs?get=CTY_NAME,ALL_VAL_MO,E_COMMODITY,E_COMMODITY_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 

surl  = surl + my_key + "&time=" + "from+2013-01" + "&COMM_LVL=HS2" 

for xxx in country_list:
    
    out_file = ".\\data"+ "\\exports\\" + xxx + "data-" + date + ".parquet"
    
    if xxx == "":
        out_file = ".\\data"+ "\\exports\\" + "TOTAL" + "data-" + date + ".parquet"
    
    
    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)

Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
3370
<Response [200]>
3010
<Response [200]>
4700
<Response [200]>
5570
<Response [200]>
5170
<Response [200]>
5081
<Response [200]>
5490
<Response [200]>
4890
<Response [200]>
4190
<Response [200]>
3330
<Response [200]>
5520
<Response [200]>
0003
<Response [200]>
0020
<Response [200]>


In [10]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])

for xxx in country_list:
    
    file = ".\\data\\exports\\" + xxx + "data-" + date + ".parquet"

    foo = pq.read_table(file).to_pandas()
    
    #recentfile = ".\\data\\exports\\" + xxx + "-" + date + "-" + "data.parquet"
    
    #foo = foo.append(pq.read_table(recentfile).to_pandas())
    
    #pq.write_table(pa.Table.from_pandas(foo), file)
    #this combines the two periods
    
    df = pd.concat([df, foo], ignore_index=True)
    
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["exports"] = df["ALL_VAL_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"exports": "sum"})
                   
total.reset_index(inplace = True)
                   
total["E_COMMODITY"] = "00"
total["E_COMMODITY_SDESC"] = "ALL PRODUCTS"
                   
df = pd.concat([df, total], ignore_index=True)                   

df.drop(["ALL_VAL_MO", "COMM_LVL", "CTY_CODE"], axis = 1, inplace = True)

df["E_COMMODITY_SDESC"] = "HS CODE " + df["E_COMMODITY"] + ", " + df["E_COMMODITY_SDESC"]

df["E_COMMODITY_SDESC"] = df["E_COMMODITY_SDESC"].replace({"HS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"})

df.set_index(["CTY_NAME","E_COMMODITY_SDESC","time"],inplace = True)

out_file = ".\\data"+ "\\top30-HS2-exports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [11]:
df.tail()

E_COMMODITY       exports
CTY_NAME E_COMMODITY_SDESC time                                
VIETNAM  ALL PRODUCTS      2025-08-01          00  1.412577e+09
                           2025-09-01          00  1.129784e+09
                           2025-10-01          00  1.547857e+09
                           2025-11-01          00  1.710346e+09
                           2025-12-01          00  1.591253e+09

In [12]:
total[total.CTY_NAME == "EUROPEAN UNION"]

,CTY_NAME,time,exports,E_COMMODITY,E_COMMODITY_SDESC
1092,EUROPEAN UNION,2013-01-01,2.025100e+10,00,ALL PRODUCTS
1093,EUROPEAN UNION,2013-02-01,2.015110e+10,00,ALL PRODUCTS
1094,EUROPEAN UNION,2013-03-01,2.300352e+10,00,ALL PRODUCTS
1095,EUROPEAN UNION,2013-04-01,2.102745e+10,00,ALL PRODUCTS
1096,EUROPEAN UNION,2013-05-01,2.246521e+10,00,ALL PRODUCTS
...,...,...,...,...,...
1243,EUROPEAN UNION,2025-08-01,3.563794e+10,00,ALL PRODUCTS
1244,EUROPEAN UNION,2025-09-01,3.820646e+10,00,ALL PRODUCTS
1245,EUROPEAN UNION,2025-10-01,3.623930e+10,00,ALL PRODUCTS
1246,EUROPEAN UNION,2025-11-01,3.332191e+10,00,ALL PRODUCTS
